# Creating Travel Agent with MultiOn and Mem0

In this recipe, we'll demonstrate how to create a travel agent that provides information and details regarding your travel. The best part is that it is capable of remembering your conversation using `mem0`.

In [ ]:
%pip install multion mem0ai

In [1]:
import os
from openai import OpenAI
from mem0 import MemoryClient
from multion.client import MultiOn

# Configuration
OPENAI_API_KEY = "sk-xxx"  # Get your OpenAI API key
MULTION_API_KEY = "xx"  # Get your MultiOn API key from https://app.multion.ai/api-keys
MEM0_API_KEY = "m0-xx"  # Get your Mem0 API key from https://app.mem0.ai/dashboard/api-keys
USER_ID = "travel_agent"

# Set up OpenAI API key
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Create Travel Agent

In [ ]:
class TravelAIAgent:
    def __init__(self):
        # Initialize Mem0 and MultiOn
        self.memory = MemoryClient(api_key=MEM0_API_KEY)
        self.multion = MultiOn(api_key=MULTION_API_KEY)

    def ask_question(self, question, user_id):
        # Fetch previous related memories
        previous_memories = self.search_memories(question, USER_ID)
        relevant_memories_text = ""
        if previous_memories:
            print("Using previous memories to enhance the search...")
            relevant_memories_text = '\n'.join(mem for mem in previous_memories)

        command = "Find travel information based on my interests:"
        prompt = f"{command}\n Question: {question} \n My preferences: {relevant_memories_text}"
        browse_result = self.multion.browse(cmd=prompt)
        answer = browse_result.message

        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": answer}
        ]

        self.memory.add(messages, user_id=user_id)
        return answer

    def search_memories(self, query, user_id):
        memories = self.memory.search(query, user_id=user_id)
        return [m['memory'] for m in memories]

### Chat with Travel Agent

In [ ]:
ai_assistant = TravelAIAgent()

while True:
    question = input("Question: ")
    if question.lower() in ['q', 'exit']:
        print("Exiting...")
        break

    answer = ai_assistant.ask_question(question, user_id=USER_ID)
    print("---------------------------------------")
    print(f"Answer: {answer}")
    print("---------------------------------------")